# Modelo de Análise de Sentimento usando SVM

No objetivo de suportar o time de marketing em melhor compreender o comportamento dos clientes. Desenvolvi este algoritmo para classificar os comentários dos reviews de maneira automatizada, para que assim o time possa tomar decisões mais rapidamente. 

Importação dos dados do dataset completo para extrair os comentários e criar os conjuntos de treinamento e teste do algoritmo

In [8]:
df = pd.read_csv('df_full.csv')

Aqui criamos o dataset com comentários positivos, filtrando aqueles com Score superior à 3

In [163]:
df_pos = df[(df.review_comment_message.notnull()) & (df.review_score > 3)] 

In [164]:
df_pos = df_pos[['review_comment_message']]
df_pos['Label'] = 'Review Positivo'

In [165]:
df_pos.head()

,review_comment_message,Label
4,Recebi bem antes do prazo estipulado.,Review Positivo
5,Parabéns lojas lannister adorei comprar pela I...,Review Positivo
13,aparelho eficiente. no site a marca do aparelh...,Review Positivo
20,"Mas um pouco ,travando...pelo valor ta Boa.\n",Review Positivo
24,"Vendedor confiável, produto ok e entrega antes...",Review Positivo


Fazemos o mesmo com os reviews negativos. Aqui usei como base aqueles com score abaixo de 3

In [166]:
df_neg = df[(df.review_comment_message.notnull()) & (df.review_score < 3)] 

In [167]:
df_neg = df_neg[['review_comment_message']]
df_neg['Label'] = 'Review Negativo'

In [168]:
df_neg.head()

,review_comment_message,Label
25,"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E...",Review Negativo
28,Péssimo,Review Negativo
40,Não gostei ! Comprei gato por lebre,Review Negativo
46,Sempre compro pela Internet e a entrega ocorre...,Review Negativo
55,Nada de chegar o meu pedido.,Review Negativo


In [169]:
df_pos.shape

(30958, 2)

In [170]:
df_neg.shape

(15637, 2)

O dataset com comentários positivos tem o dobro do tamanho do dataset com comentários negativos. Vou fazer um slice em cada um deles, juntá-los e embaralha-los para depois separar em dataset de treino e teste

In [171]:
df_pos2 = df_pos[:15000]

In [172]:
df_neg2 = df_neg[:15000]

In [173]:
df_pos_neg_full = pd.concat([df_pos2,df_neg2])

In [174]:
from sklearn.utils import shuffle
random_df = shuffle(df_pos_neg_full)

# Separação dos Dados de Teste e Treinamento

In [175]:
test_set = random_df[:15000]
test_set.shape

(15000, 2)

In [176]:
train_set = random_df[15010:]
train_set.shape

(14990, 2)

# Modelagem

In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer

import time
from sklearn import svm
from sklearn.metrics import classification_report

import pandas as pd

In [178]:
#Dados para Treinamento
trainData = train_set

#Dados para Teste
testData = test_set

#Vetorização dos comentários
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(trainData['review_comment_message'])
test_vectors = vectorizer.transform(testData['review_comment_message'])

# Classificação 

In [179]:
# Fit da classificação com SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, trainData['Label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

# Evaluation

In [180]:
print("Resultados SVC(kernel=linear)")
print("Tempo em treinamento: %fs; Tempo para Predição: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['Label'], prediction_linear, output_dict=True)
print('Positivo: ', report['Review Positivo'])
print('Negativo: ', report['Review Negativo'])

Resultados SVC(kernel=linear)
Tempo em treinamento: 7.735474s; Tempo para Predição: 4.638744s
Positivo:  {'precision': 0.9494436048907817, 'recall': 0.91475843812045, 'f1-score': 0.9317783470405825, 'support': 7555}
Negativo:  {'precision': 0.9165911151405258, 'recall': 0.9505708529214237, 'f1-score': 0.9332717921666887, 'support': 7445}


Teste e Feedback

In [370]:
review = 'gostei muito recomendo'
review_vector = vectorizer.transform([review]) 
print(classifier_linear.predict(review_vector))


['Review Positivo']


A função abaixo suporta a análise em massa de comentários

In [345]:
def analisa_review(comment):
    #Comment devem ser imputados como array
    resultado = {}
    for line in comment:
        review = line
        review_vector = vectorizer.transform([review]) 
        res = str(classifier_linear.predict(review_vector)) 
        resultado[res] = line
    return resultado

In [346]:
comentarios = df[(df.review_comment_message.notnull())]

In [347]:
comentarios = comentarios['review_comment_message'].values

In [356]:
analisa_review(comentarios)

{"['Review Positivo']": 'recebi antes do prazo.  o produto original otimo',
 "['Review Negativo']": 'Comprei duas luminárias uma amarela e outra vermelha. Só chegou a amarela. Está faltando entregar a vermelha. O status do pedido já tá como entregue.  não quero ficar no prejuízo.'}